In [1]:
import sys
sys.path.append('/Users/katiehuang/Documents/metis/projects/onl_ds5_project_2/py')
from loadpage import *
import importlib
from fictiondb import *

In [2]:
from bs4 import BeautifulSoup
import requests
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [89]:
# # Open automated Chrome with fictionDB webpage
driver = webdriver.Chrome(chromedriver)
driver.get("https://www.fictiondb.com/search/search.htm")

In [59]:
# Added sleep time in between actions
def search(author):
    author_dict={}

    driver.get("https://www.fictiondb.com/search/search.htm")
    search_bar = driver.find_element_by_xpath("//input[@name='author'][@type='text']")
    search_bar.clear()
    search_bar.send_keys(author)
    search_bar.send_keys(Keys.RETURN)
    
    # Rest for 3 seconds
    time.sleep(3)
    # Press enter in case there's pop up ad
    actions = ActionChains(driver)
    actions.send_keys(Keys.RETURN)
    actions.perform()
    
    # Search for publications first
    # If error (multiple authors page), pass
    try:
        time.sleep(1)
        actions = ActionChains(driver)
        actions.send_keys(Keys.RETURN)
        actions.perform()
        time.sleep(3)
        publication = driver.find_elements_by_xpath("//a[@itemprop='datePublished']")
        dates = [(s.text+'0').split('-') for s in publication]
        years = [int(y[-1])//10 for y in dates if int(y[-1])//10 != 0]
    except:
        print("multiple search results of author")
        pass    
    
    # Click on the author link
    try:
        author_r = ", ".join(author.split()[::-1])
        elem = driver.find_element_by_link_text(author_r)
        elem.click()
    except:
        print("can't find author")
        pass

    try:
        time.sleep(1)
        actions = ActionChains(driver)
        actions.send_keys(Keys.RETURN)
        actions.perform()
        time.sleep(2)
        publication = driver.find_elements_by_xpath("//a[@itemprop='datePublished']")
        dates = [(s.text+'0').split('-') for s in publication]
        years = [int(y[-1])//10 for y in dates if int(y[-1])//10 != 0]
    except:
        print("can't find publication")
        pass
    
    author_dict[author] = years

    return author_dict

In [52]:
time.sleep(3)
actions = ActionChains(driver)
actions.send_keys(Keys.RETURN)
actions.perform()

In [53]:
publication = driver.find_elements_by_xpath("//a[@itemprop='datePublished']")
dates = [(s.text+'0').split('-') for s in publication]
years = [int(y[-1])//10 for y in dates if int(y[-1])//10 != 0]

In [54]:
publication = driver.find_elements_by_xpath("//a[@itemprop='datePublished']")

In [55]:
dates = [(s.text+'0').split('-') for s in publication]
dates

[['Jul', '20130'],
 ['Oct', '20100'],
 ['Oct', '20060'],
 ['Oct', '20050'],
 ['May', '20010'],
 ['Nov', '20000'],
 ['Oct', '19980'],
 ['Aug', '19980'],
 ['Feb', '19980'],
 ['May', '19970'],
 ['Feb', '19960'],
 ['Mar', '19940'],
 ['0'],
 ['0'],
 ['0']]

In [56]:
years = [int(y[-1])//10 for y in dates if int(y[-1])//10 != 0]
years

[2013, 2010, 2006, 2005, 2001, 2000, 1998, 1998, 1998, 1997, 1996, 1994]

In [9]:
author = "Jane Austen"
actions = ActionChains(driver)
actions.send_keys(Keys.RETURN)
actions.perform()
author_r = ", ".join(author.split()[::-1])
elem = driver.find_element_by_link_text(author_r)
elem.click()

In [10]:
try:
    actions = ActionChains(driver)
    actions.send_keys(Keys.RETURN)
    actions.perform()
    publication = driver.find_elements_by_xpath("//a[@itemprop='datePublished']")
    dates = [(s.text+'0').split('-') for s in publication]
    years = [int(y[-1])//10 for y in dates]
except:
    print("can't find publication")
    pass

In [72]:
dates = [(s.text+'0').split('-') for s in publication if s != '']

In [69]:
years = [int(y[-1])//10 for y in dates]

In [5]:
search('Jane Austen')

{'Jane Austen': [1818,
  1817,
  1815,
  1814,
  1813,
  1811,
  1805,
  1805,
  1803,
  1790,
  1786]}

In [6]:
# search('J.K. Rowling')

In [7]:
# search('Stephen King')

In [22]:
search('James Fenimore Cooper')

can't find author


{'James Fenimore Cooper': [2017, 2012, 2012, 2010, 2009, 2006, 1841, 1826]}

In [3]:
di={}
di["jane"] = [1,2,3]

In [4]:
di

{'jane': [1, 2, 3]}

In [3]:
try:
    publication = driver.find_elements_by_xpath("//a[@itemprop='datePublished']")
    dates = [s.text.split('-') for s in publication]
    years = [int(y[-1]) for y in dates]
except:
    print("can't find publication")
    pass

In [4]:
years

[1818, 1817, 1815, 1814, 1813, 1811, 1805, 1805, 1803, 1790, 1786]

In [6]:
di={}
author="Jane"
di[author] = years

In [7]:
di

{'Jane': [1818, 1817, 1815, 1814, 1813, 1811, 1805, 1805, 1803, 1790, 1786]}

### Merge movie and book data

In [29]:
import pandas as pd
# import movie_data
movie_df = pd.read_pickle('../dump/movie_data')
# import book_data
book_df = pd.read_pickle('../dump/book_data')
found_book_df = pd.read_pickle('../dump/found_book')

In [30]:
overlap_df = pd.merge(book_df, movie_df,left_on='book_title',\
                      right_on='movie_title', how='inner')

In [4]:
# overlap_df

In [5]:
# found_book_df

In [31]:
overlap_df2 = pd.merge(found_book_df, movie_df,left_on='book_title',\
                      right_on='movie_title', how='inner')

In [10]:
# overlap_df2

In [32]:
combined_book_df = pd.concat([overlap_df,overlap_df2])

In [33]:
combined_book_df

,book_title,author,rating_value,rating_count,review_count,page,year,movie_title,rating,vote,...,runtime,director,link_d,writer,link_w,star,link_s,distributor,language,country
0,Harry Potter and the Sorcerer's Stone,J.K. Rowling,4.48,7292896.0,115556.0,309.0,1997.0,Harry Potter and the Sorcerer's Stone,7.6,654936,...,159.0,Chris Columbus,/name/nm0001060/,"[J.K. Rowling, Steve Kloves]","[/name/nm0746830/, /name/nm0460141/]","[Daniel Radcliffe, Rupert Grint, Richard Harris]","[/name/nm0705356/, /name/nm0342488/, /name/nm0...",WarnerBros.,[English],"[UK, USA]"
1,Harry Potter and the Chamber of Secrets,J.K. Rowling,4.43,2821010.0,54542.0,341.0,1998.0,Harry Potter and the Chamber of Secrets,7.4,552861,...,174.0,Chris Columbus,/name/nm0001060/,"[J.K. Rowling, Steve Kloves]","[/name/nm0746830/, /name/nm0460141/]","[Daniel Radcliffe, Rupert Grint, Emma Watson]","[/name/nm0705356/, /name/nm0342488/, /name/nm0...",WarnerBros.,[English],"[UK, USA, Germany]"
2,Harry Potter and the Prisoner of Azkaban,J.K. Rowling,4.57,2908354.0,57262.0,435.0,1999.0,Harry Potter and the Prisoner of Azkaban,7.9,550532,...,142.0,Alfonso Cuarón,/name/nm0190859/,"[J.K. Rowling, Steve Kloves]","[/name/nm0746830/, /name/nm0460141/]","[Daniel Radcliffe, Emma Watson, Rupert Grint]","[/name/nm0705356/, /name/nm0914612/, /name/nm0...",WarnerBros.,[English],"[UK, USA]"
3,Harry Potter and the Goblet of Fire,J.K. Rowling,4.56,2676621.0,47961.0,734.0,2000.0,Harry Potter and the Goblet of Fire,7.7,546725,...,166.0,Mike Newell,/name/nm0001565/,"[Steve Kloves, J.K. Rowling]","[/name/nm0460141/, /name/nm0746830/]","[Daniel Radcliffe, Emma Watson, Rupert Grint]","[/name/nm0705356/, /name/nm0914612/, /name/nm0...",WarnerBros.,"[English, French]","[UK, USA]"
4,Harry Potter and the Order of the Phoenix,J.K. Rowling,4.50,2585450.0,43737.0,870.0,2003.0,Harry Potter and the Order of the Phoenix,7.5,509436,...,138.0,David Yates,/name/nm0946734/,"[Michael Goldenberg, J.K. Rowling]","[/name/nm0325533/, /name/nm0746830/]","[Daniel Radcliffe, Emma Watson, Rupert Grint]","[/name/nm0705356/, /name/nm0914612/, /name/nm0...",WarnerBros.,[English],"[UK, USA]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,Little Caesar,W.R. Burnett,3.67,246.0,35.0,160.0,1929.0,Little Caesar,7.2,12030,...,79.0,Mervyn LeRoy,/name/nm0503777/,"[W.R. Burnett, Francis Edward Faragoh, 2 more ...","[/name/nm0122446/, /name/nm0267020/, fullcredi...","[Edward G. Robinson, Douglas Fairbanks Jr., Gl...","[/name/nm0000064/, /name/nm0001195/, /name/nm0...",FirstNationalPictures,[English],[USA]
655,How to Make an American Quilt,Whitney Otto,3.62,4902.0,328.0,254.0,1991.0,How to Make an American Quilt,6.3,10670,...,117.0,Jocelyn Moorhouse,/name/nm0602104/,"[Whitney Otto, Jane Anderson]","[/name/nm0653307/, /name/nm0026862/]","[Winona Ryder, Ellen Burstyn, Anne Bancroft]","[/name/nm0000213/, /name/nm0000995/, /name/nm0...",UniversalPictures,[English],[USA]
656,Fifty Shades Darker,E.L. James,3.84,761148.0,29215.0,532.0,2011.0,Fifty Shades Darker,4.6,91676,...,131.0,James Foley,/name/nm0001226/,"[Niall Leonard, E.L. James]","[/name/nm0502727/, /name/nm1093317/]","[Dakota Johnson, Jamie Dornan, Eric Johnson]","[/name/nm0424848/, /name/nm1946193/, /name/nm0...",UniversalPictures,[English],"[USA, China, Japan]"
657,Bless the Child,Cathy Cash Spellman,3.80,607.0,54.0,608.0,1993.0,Bless the Child,5.1,13867,...,107.0,Chuck Russell,/name/nm0751080/,"[Cathy Cash Spellman, Thomas Rickman, 2 more c...","[/name/nm0817697/, /name/nm0725564/, fullcredi...","[Kim Basinger, Jimmy Smits, Rufus Sewell]","[/name/nm0000107/, /name/nm0001751/, /name/nm0...",ParamountPictures,[English],"[USA, Germany]"


In [34]:
# Create list of author
author_list = list(combined_book_df.author)

In [35]:
author_list

['J.K. Rowling',
 'J.K. Rowling',
 'J.K. Rowling',
 'J.K. Rowling',
 'J.K. Rowling',
 'Stephenie Meyer',
 'J.K. Rowling',
 'Jane Austen',
 'Jane Austen',
 'Suzanne Collins',
 'Dan Brown',
 'Roald Dahl',
 'E.B. White',
 'E.B. White',
 'Harper Lee',
 'Roald Dahl',
 'Christopher Paolini',
 'Louisa May Alcott',
 'Louisa May Alcott',
 'Louisa May Alcott',
 'Louisa May Alcott',
 'Charlotte Brontë',
 'Charlotte Brontë',
 'Charlotte Brontë',
 'Charles Dickens',
 'Charles Dickens',
 'F. Scott Fitzgerald',
 'F. Scott Fitzgerald',
 'F. Scott Fitzgerald',
 'Nicholas Sparks',
 'S.E. Hinton',
 'Margaret Mitchell',
 'Dan Brown',
 'Philip Pullman',
 'Jane Austen',
 'Gail Carson Levine',
 'J.M. Barrie',
 'J.M. Barrie',
 'Frances Hodgson Burnett',
 'Frances Hodgson Burnett',
 'Frances Hodgson Burnett',
 'Katherine Paterson',
 'Arthur Golden',
 'Meg Cabot',
 'Helen Fielding',
 'William Goldman',
 'Michael Crichton',
 'Jane Austen',
 'Ann Brashares',
 'Nicholas Sparks',
 'Stephen King',
 'John Steinbeck',

In [39]:
# Remove duplicates by converting to set
author_list_set = set(author_list)

In [40]:
# author_list_s

In [41]:
len(author_list_set)

876

In [43]:
import pickle
with open('../dump/author_list_set', 'wb') as f:
    pickle.dump(author_list_set, f)

In [1]:
import pandas as pd
author_list_set = pd.read_pickle('../dump/author_list_set')

In [26]:
author_list_test = {'Oscar Wilde',
 'Natalie Babbitt',
 'Stephen King'}

In [2]:
import sys
sys.path.append('/Users/katiehuang/Documents/metis/projects/onl_ds5_project_2/py')
from loadpage import *
import importlib
from fictiondb import *
from bs4 import BeautifulSoup
import requests
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [23]:
# # Open automated Chrome with fictionDB webpage
driver = webdriver.Chrome(chromedriver)
driver.get("https://www.fictiondb.com/search/search.htm")

In [25]:
c = 0
author_book_list=[]
for author in author_list_set:
    c += 1
    print(c,author)
    result = search(author)
    author_book_list.append(result)

1 Anna Todd
can't find author
2 Alex Flinn
can't find author
3 James Fenimore Cooper
can't find author
4 Amy Daws
5 John Bellairs
can't find author
6 Knut Hamsun
can't find author
7 Agatha Christie
8 Nicholas Shakespeare
can't find author
9 Maurice G. Dantec
can't find author
10 Trevanian
can't find author
11 Edith Wharton
12 Ron Hansen
can't find author
13 Max Ehrlich
14 F. Scott Fitzgerald
can't find author
15 Davis Grubb
can't find author
16 Dr US President Princess Sonja Miranda Allister MD
can't find author
17 Scott Heim
can't find author
18 Truman Capote
can't find author
19 Neal Stephenson
20 Cressida Cowell
can't find author
21 William P. Kennedy
can't find author
22 Walter Scott
can't find author
23 Stephenie Meyer
can't find author
24 Joseph Finder
25 E.L. Doctorow
can't find author
26 Wilson Rawls
can't find author
27 Philip K. Dick
can't find author
28 Steven Piziks
can't find author
29 John     Nichols
30 Charles Webb
31 Anne Holm
can't find author
32 Jay Bell
can't find a

266 Eoin Colfer
267 Peter Farrelly
can't find author
268 Julio Cortázar
can't find author
269 Lauran Paine
270 Judith Pella
271 Silvio Raffo
can't find author
272 Christine Warren
can't find author
273 Henryk Sienkiewicz
can't find author
274 Ross Lockridge Jr.
can't find author
275 Shauna Cross
can't find author
276 Joseph Hayes
277 W.H. Hudson
can't find author
278 Frederic P.  Miller
can't find author
279 Isaac Asimov
280 Dan Wells
can't find author
281 George Barr McCutcheon
can't find author
282 Jane Hamilton
283 Steve Martin
284 Renee Pawlish
can't find author
285 Tracy Anne Warren
can't find author
286 Willie Morris
can't find author
287 Richard Russo
288 William J. Mann
can't find author
289 David Mack
290 Leo Tolstoy
291  William Peter Blatty
can't find author
292 John Updike
can't find author
293 Louis Sachar
can't find author
294 Dave Eggers
295 Scott Turow
can't find author
296 Stephen Coonts
297 Richard Atwater
can't find author
298 David Cameron
can't find author
299 Raym

WebDriverException: Message: chrome not reachable
  (Session info: chrome=87.0.4280.141)


In [108]:
# Save to pickle
import pickle
with open('../dump/author_book_list', 'wb') as f:
    pickle.dump(author_book_list, f)

In [109]:
import pandas as pd
author_book_list = pd.read_pickle('../dump/author_book_list')

In [111]:
c = len(author_book_list)
c

747

In [33]:
# search("James Fenimore Cooper")

In [112]:
done_author=[]
for dic in author_book_list:
    for key, value in dic.items():
        print(key)
        done_author.append(key)

Anna Todd
Alex Flinn
James Fenimore Cooper
Amy Daws
John Bellairs
Knut Hamsun
Agatha Christie
Nicholas Shakespeare
Maurice G. Dantec
Trevanian
Edith Wharton
Ron Hansen
Max Ehrlich
F. Scott Fitzgerald
Davis Grubb
Dr US President Princess Sonja Miranda Allister MD
Scott Heim
Truman Capote
Neal Stephenson
Cressida Cowell
William P. Kennedy
Walter Scott
Stephenie Meyer
Joseph Finder
E.L. Doctorow
Wilson Rawls
Philip K. Dick
Steven Piziks
John     Nichols
Charles Webb
Anne Holm
Jay Bell
Frederick Kohner
Mark Teague
Philip MacDonald
Eric Hodgins
Louis de Bernières
Marsha Alexander
William F. Nolan
Andre Dubus III
Winifred Watson
Jenny Han
Madeleine L'Engle
Donn Pearce
Tim Sandlin
Kate DiCamillo
Martina Boone
Henry James
Robert Sheckley
Denton J. Tipton
Chaim Potok
Elizabeth Essex
Andrew Wellman
Ayn Rand
Desmond Bagley
Kurt Vonnegut Jr.
Daniel Wallace
Ruth Kyser
Gordon Korman
Louis Begley
Jim Stovall
Lois Lowry
Lloyd C. Douglas
Tim Tharp
Tom Clancy
Vladimir Nabokov
M.L. Stedman
Daniel Woodrel

In [113]:
len(done_author)

747

In [114]:
left_author_set = author_list_set.difference(set(done_author))

In [115]:
len(left_author_set)

129

In [47]:
for x in author_book_list:
    for author, year in x.items():
        if author == "Elle Kennedy":
            print(year)

[2020, 2020, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2018, 2017, 2016, 2016, 2016, 2016, 2016, 2016, 2015, 2015, 2015, 2015, 2015, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2012, 2012, 2012, 2012, 2012, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2010, 2010, 2010, 2010, 2009, 2009, 2009, 2009, 2008, 2008, 2008, 2008, 2007, 1975]


In [116]:
%run -i '../py/fictiondb.py'

In [117]:
for author in left_author_set:
    c += 1
    print(c,author)
    result = search(author)
    author_book_list.append(result)

748 Richard McKenna
749 Gillian Flynn
can't find author
750 Lisa Kleypas
can't find author
751 Michael Ondaatje
can't find author
752 Ken Bruen
753 Nick Hornby
can't find author
754 Uzodinma Iweala
can't find author
755 Garth Stein
can't find author
756 Horace McCoy
757 William  Kennedy
758 James Haskins
can't find author
759 Zane Grey
760 Barry Gifford
can't find author
761 John D. MacDonald
can't find author
762 Lisa Oliver
763 Jim Carroll
can't find author
764 Cornelius Ryan
can't find author
765 Lisa Grunwald
can't find author
766 Ann Brashares
can't find author
767 Jonathan Lethem
768 Henry Farrell
can't find author
769 Blake Nelson
770 Paul Brodeur
can't find author
771 Ken Kesey
772 Irène Némirovsky
can't find author
773 Neil Gaiman
774 S.J. Watson
can't find author
775 Aleksandar Hemon
can't find author
776 Jonathan Harr
can't find author
777 Leon Uris
can't find author
778 Harper Lee
779 Jack Schaefer
can't find author
780 Serena Valentino
781 Gerald Kersh
can't find author
78

In [75]:
# search('Norman Partridge')

In [118]:
len(author_book_list)

876

In [119]:
# Save to pickle
import pickle
with open('../dump/author_book_list', 'wb') as f:
    pickle.dump(author_book_list, f)

In [120]:
import pandas as pd
author_book_list = pd.read_pickle('../dump/author_book_list')

In [121]:
len(author_book_list)

876

In [134]:
author_book_list_test = []
for author in author_book_list:
    headers = ['author','years']
    for key, value in author.items():
        author = key
        years = value
    
    author_dict = dict(zip(headers,[author,years]))
    
    author_book_list_test.append(author_dict)
    
len(author_book_list_test)

876

In [136]:
author_book_df = pd.DataFrame(author_book_list_test)
author_book_df

,author,years
0,Anna Todd,"[2020, 2018, 2018, 2017, 2017, 2016, 2016, 201..."
1,Alex Flinn,"[2020, 2019, 2017, 2015, 2015, 2013, 2012, 201..."
2,James Fenimore Cooper,"[2017, 2012, 2012, 2010, 2009, 2006, 1841, 1826]"
3,Amy Daws,"[2020, 2019, 2019, 2018, 2018, 2018, 2018, 201..."
4,John Bellairs,"[2020, 2015, 2009, 2004, 2004, 2004, 2004, 200..."
...,...,...
871,Frances Hodgson Burnett,"[2017, 2011, 2009, 2008, 2008, 2008, 1971, 191..."
872,Nicola Yoon,"[2021, 2016, 2015]"
873,Michelle Zink,"[2020, 2018, 2016, 2015, 2015, 2015, 2013, 201..."
874,Julia Walton,"[2021, 2017]"


In [137]:
# Save to pickle
import pickle
with open('../dump/author_book_data', 'wb') as f:
    pickle.dump(author_book_df, f)